Read in Argoverse HD map and COLMAP model

In [1]:
from transforms import affine_matrix_from_points
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
from scipy.spatial import transform
import matplotlib
import matplotlib.pyplot as plt
from visualize_30hz_benchmark_data_on_map import DatasetOnMapVisualizer
from argoverse.map_representation.map_api import ArgoverseMap
import logging
import numpy as np
import numpy.linalg as LA

# path to the argoverse dataset
tracking_dataset_dir = '../argoverse-tracking/sample/'

# log_index determines which log in the argoverse dataset directory to use
log_index = 0

# load argoverse HD map
argoverse_loader = ArgoverseTrackingLoader(tracking_dataset_dir)
log_id = argoverse_loader.log_list[log_index]
argoverse_data = argoverse_loader[log_index]

am = ArgoverseMap()
city_name = argoverse_data.city_name
dataset_dir = tracking_dataset_dir
experiment_prefix = 'visualization_demo'
use_existing_files = True

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
domv = DatasetOnMapVisualizer(dataset_dir, experiment_prefix, use_existing_files=use_existing_files, log_id=argoverse_data.current_log)

# colmap_output_dir points to the results from COLMAP
colmap_output_dir = 'reconstruction/sparse/' + log_id + '/'
# detection results directory
detection_output_dir = './focal_detection/'
# where to save computed traffic sign locations
projection_output_dir = './'


#read in colmap results
poses_colmap = dict()   #dict that queries camera poses with image name
points_colmap = dict()  #dict that queries colmap feature points IDs with image name
id2name_colmap = dict() #dict that queries image name with colmap image IDs
points3D = dict()       #dict that queries colmap points' 3D coordinates with colmap point IDs

with open(colmap_output_dir + 'images.txt') as f:
    lines = f.read().splitlines()
for i,line in enumerate(lines):
    if line[0] == '#':
        continue
    else:
        if i % 2 == 0:
            fields = line.split(' ')
            image_name = fields[-1]
            image_id = int(fields[0])
            quat = fields[1:5]
            quatanion = np.array([float(q) for q in quat])
            trans = fields[5:8]
            translation = np.array([float(t) for t in trans])
            camera_id = int(fields[8])
            poses_colmap[image_name] = [quatanion, translation, image_id, camera_id]
            id2name_colmap[image_name] = image_id
        else:
            fields = line.split(' ')
            points_2d = np.array([float(pt) for pt in fields])
            points_colmap[image_name] = points_2d
            
with open(colmap_output_dir + 'points3D.txt') as f:
    lines = f.read().splitlines()
for i,line in enumerate(lines):
    if line[0] == '#':
        continue
    else: 
            fields = line.split(' ')
            points_attr = np.array([float(pt) for pt in fields])
            points3D[points_attr[0]] = np.array([points_attr[1],points_attr[2],points_attr[3]])
            




INFO:argoverse.data_loading.synchronization_database:Building SynchronizationDB
INFO:root:syncronizing camera and lidar sensor...
INFO:argoverse.data_loading.vector_map_loader:Loaded root: ArgoverseVectorMap
INFO:argoverse.data_loading.vector_map_loader:Loaded root: ArgoverseVectorMap


Read in traffic sign detection results and pre-stored traffic sign positions from database

In [2]:
import cv2 
import os
import numpy as np

# read in detection results
boxes = np.load(detection_output_dir + log_id + '.npz',allow_pickle=True)
boxes_images_list = [full_name.split('/')[-1] for full_name in boxes['input_image_names']]   
order = np.argsort(boxes_images_list)
boxes_images_list = np.array(boxes_images_list)[order]    # image names for the detection bounding boxes
boxes_boxes_list = boxes['bbox_list'][order]              # detection bounding boxes
boxes_scores_list = boxes['score_list'][order]            # confidence scores for the detection bounding boxes
boxes_class_list = boxes['pred_classes_list'][order]      # detected categories for the detection bounding boxes

# read in stored traffic sign postions and the 3D-to-3D similarity transformation
database = np.load(log_id + '_focal_projected.npz',allow_pickle=True)
argo_sign_poses = database['argo_sign_poses']
categories = database['categories']
colmap2argo = database['colmap2argo']
argo2colmap = database['argo2colmap']
M = colmap2argo
M_inv = argo2colmap

Some utility functions

In [3]:
def check_inside(x1,y1,x2,y2,x,y):
    return x1 <= x and x <= x2 and y1 <= y and y <=y2

# discard these signs that are expected too far away from the vehicle
def checkDistance(image_name, sign_position):
    q = poses_colmap[image_name][0]
    quat = [q[1], q[2], q[3], q[0]]
    R = transform.Rotation.from_quat(quat)
    t = poses_colmap[image_name][1]
    vehicle_position_colmap = - R.as_matrix().T.dot(t)
    vehicle_position_argo = M.dot(np.append(vehicle_position_colmap, [1]))
    L2_distance = np.sqrt(np.sum(np.square(vehicle_position_argo[:3] - sign_position)))
    return L2_distance

For each traffic sign stored in the databse, get its expected 2D postion in each frame and count the total number
of times it has been successfully detected

In [5]:
# store the number of successful detections for each prestored traffic sign in the database
number_of_occurences = np.zeros(argo_sign_poses.shape[0])   

calib = argoverse_data.get_calibration("ring_front_center")

for idx in range(len(argoverse_data.image_list_sync['ring_front_center'])):
    city_to_egovehicle_se3 = argoverse_data.get_pose(idx)
    
    # compute the 3D-to-2D projection from HD map coordinate to image view
    img_fc = argoverse_data.image_list_sync['ring_front_center'][idx]
    img_fc = img_fc.split('/')
    image_name = img_fc[-2] + '/' + img_fc[-1]
    q = poses_colmap[image_name][0]
    quat = [q[1], q[2], q[3], q[0]]
    R = transform.Rotation.from_quat(quat)
    t = poses_colmap[image_name][1]
    E = np.zeros((3,4))
    E[:3,:3] = R.as_matrix()
    E[:3,3] = t
    
    # project each pre-stored sign in the database
    for i in range(argo_sign_poses.shape[0]):
        # Start projection only when the vehicle is close enough to the sign 
        distance_2_sign = checkDistance(image_name, argo_sign_poses[i,:])
        if distance_2_sign > 50:
            continue
        pts_argo = np.append(argo_sign_poses[i,:], [1]).reshape(4,1)
        pts_colmap = M_inv @ pts_argo
        pts_uv = E @ pts_colmap
        K = np.array([[1352.38, 0, 960], [0, 1352.38, 600], [0, 0, 1]]) #camera intrinsics
        uv = K @ pts_uv
        uv = uv/uv[2]
        uv = np.squeeze(uv)
        
        # check whether the position in the image view lies within any detection bounding box of that category
        detection_idx = np.where(boxes_images_list==img_fc[-1])[0].item()
        for j,box in enumerate(boxes_boxes_list[detection_idx]):
            margin = 5       # The width that we want to expand each bounxing boxes with
            x1 = int(box[0]) # - margin
            y1 = int(box[1]) # - margin
            x2 = int(box[2]) # + margin
            y2 = int(box[3]) # + margin
            if check_inside(x1, y1, x2, y2, uv[0], uv[1]) and boxes_class_list[detection_idx][j]==categories[i]:
                number_of_occurences[i] += 1

print(number_of_occurences)

[ 3. 86. 25. 20. 39. 10. 36.  1. 31.  5.]


Remove the old signs that are not detected in the sequnce, and update the npz file accrodingly

In [6]:
valid = number_of_occurences != 0
argo_sign_poses = argo_sign_poses[valid]
categories = categories[valid]

np.savez(projection_output_dir + log_id+'_after_removal.npz', 
         argo_sign_poses=argo_sign_poses, 
         categories=categories,
         colmap2argo=colmap2argo,
         argo2colmap=argo2colmap,
        )